# bondret

> Retrieve and process data from WRDS Bond Returns File 

Since this is a proprietary dataset, in the documentation below, I can not show any of the data that is being retrieved/generated (will show only column names).

In [ ]:
#| default_exp wrds.bondret

In [ ]:
#|exports
from __future__ import annotations
from typing import List

import pandas as pd
import numpy as np

import pandasmore as pdm
from finsets.wrds import wrds_api

In [ ]:
#| exports
PROVIDER = 'Wharton Research Data Services (WRDS)'
URL = 'https://wrds-www.wharton.upenn.edu/pages/get-data/wrds-bond-returns/'
LIBRARY = 'wrdsapps'
TABLE = 'bondret'
LINK_TABLE = 'bondcrsp_link'
FREQ = 'M'
MIN_YEAR = 2002
MAX_YEAR = None
ENTITY_ID_IN_RAW_DSET = 'permno'
ENTITY_ID_IN_CLEAN_DSET = 'permno'
TIME_VAR_IN_RAW_DSET = 'date'
TIME_VAR_IN_CLEAN_DSET = f'{FREQ}date'

In [ ]:
#| export
def list_all_vars() -> pd.DataFrame:
    "Collects names of all available variables from WRDS f`{LIBRARY}.{TABLE}`."

    try:
        db = wrds_api.Connection()
        df = db.describe_table(LIBRARY,TABLE).assign(wrds_library=LIBRARY, wrds_table=TABLE)
    finally:
        db.close()

    return df[['name','type','wrds_library','wrds_table']].copy()

In [ ]:
#| eval: false
all_vars = list_all_vars()

In [ ]:
#| eval: false
all_vars.head()

,name,type,wrds_library,wrds_table
0,date,DATE,wrdsapps,bondret
1,issue_id,DOUBLE_PRECISION,wrdsapps,bondret
2,cusip,VARCHAR(9),wrdsapps,bondret
3,bond_sym_id,VARCHAR(14),wrdsapps,bondret
4,bsym,VARCHAR(12),wrdsapps,bondret


In [ ]:
#| export
def default_raw_vars():
    """Defines default variables used in `get_raw_data` if none are specified."""

    return ['date','cusip','maturity', 'ncoups', 'price_eom', 'ret_eom', 'yield', 'coupon', 'remcoups', 'amount_outstanding', 'principal_amt']            

In [ ]:
print(default_raw_vars())

['date', 'cusip', 'maturity', 'ncoups', 'price_eom', 'ret_eom', 'yield', 'coupon', 'remcoups', 'amount_outstanding', 'principal_amt']


In [ ]:
#| export
def parse_varlist(vars: List[str]=None,
                  required_vars = [],
                  ) -> str:
    """Validates that `vars` are available in `{LIBRARY}.{TABLE}` table and adds a. prefixes to variable names to feed into an SQL query"""

    # Get all available variables and add suffixes needed for the SQL query
    suffix_mapping = {TABLE: 'a.'}
    all_avail_vars = list_all_vars().drop_duplicates(subset='name',keep='first').copy()
    all_avail_vars['w_prefix'] = all_avail_vars.apply(lambda row: suffix_mapping[row['wrds_table']] + row['name'] , axis=1)

    if vars == '*': return ','.join(list(all_avail_vars['w_prefix']))
    
    # Add required vars to requested vars
    if vars is None: vars = default_raw_vars()
    vars_to_get =  required_vars + [x for x in list(set(vars)) if x not in required_vars]

    # Validate variables to be downloaded (make sure that they are in the target database)
    invalid_vars = [v for v in vars_to_get if v not in list(all_avail_vars.name)]
    if invalid_vars: raise ValueError(f"These vars are not in the database: {invalid_vars}") 

    # Extract information on which variable comes from which wrds table, so we know what prefix to use
    vars_to_get = pd.DataFrame(vars_to_get, columns=['name'])
    get_these = vars_to_get.merge(all_avail_vars, how = 'left', on = 'name')
        
    return ','.join(list(get_these['w_prefix']))

In [ ]:
#| eval: false
parse_varlist()

'a.ncoups,a.amount_outstanding,a.yield,a.remcoups,a.ret_eom,a.date,a.cusip,a.principal_amt,a.price_eom,a.coupon,a.maturity'

In [ ]:
#| export
def get_raw_data(
        vars: List[str]=None, # If None, downloads `default_raw_vars`; use '*' to get all available variables
        required_vars: List[str]=['cusip','date'], #list of variables that will get downloaded, even if not in `vars`
        nrows: int=None, #Number of rows to download. If None, full dataset will be downloaded
        start_date: str=None, # Start date in MM/DD/YYYY format
        end_date: str=None #End date in MM/DD/YYYY format
) -> pd.DataFrame:
    """Downloads `vars` from `start_date` to `end_date` from WRDS `{LIBRARY}.{TABLE}` library and adds PERMNO and PERMCO"""
 
    wrds_api.validate_dates([start_date, end_date])
    vars = parse_varlist(vars, required_vars=required_vars)

    sql_string=f"""SELECT c.permno as permno, c.permco as permco, {vars}
                    FROM {LIBRARY}.{TABLE} AS a
                    INNER JOIN {LIBRARY}.{LINK_TABLE} AS c ON a.cusip = c.cusip 
                    WHERE date BETWEEN c.link_startdt AND COALESCE(c.link_enddt, CURRENT_DATE)
                """
    if start_date is not None: sql_string += r" AND date >= %(start_date)s"
    if end_date is not None: sql_string += r" AND date <= %(end_date)s"
    if nrows is not None: sql_string += r" LIMIT %(nrows)s"
    
    return wrds_api.download(sql_string,
                             params={'start_date':start_date, 'end_date':end_date, 'nrows':nrows})

In [ ]:
#| eval: false
raw = get_raw_data(nrows=1000)

In [ ]:
#| eval: false
raw.head(0)

,permno,permco,cusip,date,ncoups,amount_outstanding,yield,remcoups,ret_eom,principal_amt,price_eom,coupon,maturity


In [ ]:
#| export
def process_raw_data(
        df: pd.DataFrame=None,  # Must contain `permno` and `datadate` columns   
        clean_kwargs: dict={},  # Params to pass to `pdm.setup_panel` other than `panel_ids`, `time_var`, and `freq`
) -> pd.DataFrame:
    """Applies `pandasmore.setup_panel` to `df`"""

    # Change some variables to categorical
    for col in ['permno','permco']:
        if col in df.columns:
            df[col] = df[col].astype('Int64').astype('category')

    for col in ['cusip']:
        if col in df.columns:
            df[col] = df[col].astype('string').astype('category')

    # Set up panel structure
    df = pdm.setup_panel(df, 
                         drop_index_duplicates=False, # Multiple bonds (cusips) per permno per date
                         panel_ids=ENTITY_ID_IN_RAW_DSET, time_var=TIME_VAR_IN_RAW_DSET, freq=FREQ, 
                         panel_ids_toint=False, 
                         **clean_kwargs)
    return df 

In [ ]:
#| eval: false
df_clean = process_raw_data(raw)

In [ ]:
#| eval: false
df_clean.head()

date     dtdate permco      cusip  ncoups  \
permno Mdate                                                      
26403  2002-07  2002-07-31 2002-07-31  20587  00072AAA8     2.0   
       2002-08  2002-08-31 2002-08-31  20587  00072AAA8     2.0   
       2002-10  2002-10-31 2002-10-31  20587  00072AAA8     2.0   
       2002-11  2002-11-30 2002-11-30  20587  00072AAA8     2.0   
       2003-02  2003-02-28 2003-02-28  20587  00072AAA8     2.0   

                amount_outstanding     yield  remcoups   ret_eom  \
permno Mdate                                                       
26403  2002-07             59905.0  0.063246      14.0       NaN   
       2002-08             59905.0  0.061086      13.0  0.015265   
       2002-10             59905.0  0.068340      13.0       NaN   
       2002-11             59905.0  0.056472      13.0  0.062946   
       2003-02             59905.0  0.052255      12.0       NaN   

                principal_amt   price_eom  coupon    maturity  
permno Mdate                                                   
26403  2002-07         1000.0  117.597667   9.645  2009-02-15  
       2002-08         1000.0  118.601000   9.645  2009-02-15  
       2002-10         1000.0  114.125000   9.645  2009-02-15  
       2002-11         1000.0  120.644000   9.645  2009-02-15  
       2003-02         1000.0  122.421000   9.645  2009-02-15

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()